# Fast IP Range Assessment
- Author: Aidan Marlin
- Date: 20221010
- Version: 1.0
- Description: Jupyter Notebook to automate the execution of key Windows assessment scans

In [ ]:
import socket
import os

###
### Change these variables if necessary
###
### Specify IP like 192.168.0.1
target_ip = "192.168.0.1"
vpn = True

# Check this before running rest of notebook
router_ip = os.popen('ip r | grep default | choose 2').read().rstrip()
router_mac = os.popen("arp -n | grep \"" + router_ip + " \" | choose 2").read().rstrip()

if vpn == True:
    interface = os.popen('nmcli connection show --active | grep -E "tun|tap|wireguard" | choose 0').read().rstrip()
else:
    interface = os.popen('ip r | grep default | choose 4').read().rstrip()

###
### No need to modify from here
###

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

!mkdir -p tools/masscan
!mkdir -p tools/nmap
!mkdir -p tools/puppeteer

def read_file(location):
    try:
        with open(location) as f:
            text = f.read()
            return text
    except:
        return False

# Local network
print("Router IP: " + router_ip)
print("Router MAC: " + router_mac)
print("Interface: " + interface)


# Target

In [ ]:
print(target_ip)


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Discovery

## Ping Scan

In [ ]:
!time nmap -sn -T5 {target_ip} -v0 -oA tools/nmap/internal_ping_scan


In [ ]:
print(read_file("tools/nmap/internal_ping_scan.gnmap"))


## Fast TCP Port Scan

In [ ]:
!sudo time masscan --top-ports 100 --interface {interface} --rate=1000 -oB tools/masscan/tcp_top_100.bin {target_ip}/32


In [ ]:
!sudo masscan --readscan tools/masscan/tcp_top_100.bin -oG tools/masscan/tcp_top_100.grep
!sudo masscan --readscan tools/masscan/tcp_top_100.bin -oJ tools/masscan/tcp_top_100.json
!sudo masscan --readscan tools/masscan/tcp_top_100.bin -oX tools/masscan/tcp_top_100.xml


In [ ]:
print(read_file("tools/masscan/tcp_top_100.grep"))


## nbtscan

In [ ]:
!time docker run -it --rm booyaabes/kali-linux-full "/usr/bin/nbtscan" {target_ip}


## impacket (lookupsid.py)

In [ ]:
%%bash -s "$target_ip"
time docker run --rm rflathers/impacket lookupsid.py -no-pass anonymous@$1


## crackmapexec

In [ ]:
!time docker run -it --rm booyaabes/kali-linux-full "/usr/bin/crackmapexec" smb {target_ip}


## smbmap

In [ ]:
!time docker run -it --rm booyaabes/kali-linux-full "/usr/bin/smbmap" -u anonymous -H {target_ip}


## nmblookup

In [ ]:
!time docker run -it --rm booyaabes/kali-linux-full "/usr/bin/nmblookup" -A {target_ip}


## nmap (smb-vuln*)

In [ ]:
!time nmap --script smb-vuln\* {target_ip}


## enum4linux

In [ ]:
!time enum4linux {target_ip}


## Notify

In [ ]:
!/home/user/git/maxos/scripts/telegram_notify.sh -a -m "PT Windows Assessment complete for {target_ip}."


# End Time

In [ ]:
!date
